Adjust workflow
*change load classes to the utils
* add in the youtube download to utils XX
* add changes for above two changes the main.py XX
* change main.py to train.py so you can full automate to CLI easiest
* add in the classes to the labels visualizations XX
* add uses to it 
* add to hugging face/Snap AR
* attempt to use cv2 to see when basketballs intersect with rims XX

images outside of youtube nba footage come from: [text](https://stock.adobe.com/search?filters%5Bcontent_type%3Aphoto%5D=1&filters%5Bcontent_type%3Aillustration%5D=1&filters%5Bcontent_type%3Azip_vector%5D=1&filters%5Bcontent_type%3Avideo%5D=1&filters%5Bcontent_type%3Atemplate%5D=1&filters%5Bcontent_type%3A3d%5D=1&filters%5Bcontent_type%3Aaudio%5D=0&filters%5Binclude_stock_enterprise%5D=0&filters%5Bis_editorial%5D=0&filters%5Bfree_collection%5D=0&filters%5Bcontent_type%3Aimage%5D=1&k=basketball+hoop&order=relevance&safe_search=1&limit=100&search_page=21&search_type=pagination&get_facets=0)

In [1]:
import torch
from torch import nn

# Note: this notebook requires torch >= 1.10.0
print(torch.__version__)
print("CUDA available: ", torch.cuda.is_available())


# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

2.1.2+cu118
CUDA available:  True


'cuda'

In [2]:
import os

def create_directory(dir_path):
    """Create a directory if it does not exist."""
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

#create going_modular repository
create_directory("going_modular")

In [ ]:
# Download required files from torchvision
import requests

def download_files(urls):
    for url in urls:
        response = requests.get(url)
        if response.status_code == 200:
            with open(url.split("/")[-1], 'wb') as file:
                file.write(response.content)
        else:
            print(f"Failed to download {url}. Status code: {response.status_code}")

urls = [
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py",
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py",
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py",
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py",
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py"
]
download_files(urls)

In [3]:
%%writefile going_modular/utils.py
#!pip install roboflow


#from roboflow import Roboflow
#rf = Roboflow(api_key="htpcxp3XQh7SsgMfjJns")
#project = rf.workspace("ai-79z1a").project("basketball_child")
#dataset = project.version(6).download("coco-segmentation")


from roboflow import Roboflow
import torch
import requests
import yt_dlp
import os

def download_videos_from_youtube(video_urls, output_path):
    """
    Downloads videos from YouTube.

    Args:
    video_urls (list): List of YouTube video URLs.
    output_path (str): Directory where videos will be saved.

    Returns:
    tuple: A tuple containing lists of successful and failed downloads.
    """

    ydl_opts = {
        'format': 'best',
        'outtmpl': output_path + '/%(title)s.%(ext)s',
        'quiet': True
    }

    failed_downloads = []
    successful_downloads = []

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        for url in video_urls:
            try:
                ydl.download([url])
                print(f"Successfully downloaded {url}")
                successful_downloads.append(url)
            except Exception as e:
                print(f"Failed to download {url}: {e}")
                failed_downloads.append(url)

    return successful_downloads, failed_downloads


def get_device():
    return "cuda" if torch.cuda.is_available() else "cpu"


def get_project(api_key, workspace, project_name, version):
    rf = Roboflow(api_key=api_key)
    project = rf.workspace(workspace).project(project_name)
    dataset = project.version(version).download("coco-segmentation")
    return dataset

def download_files(urls):
    for url in urls:
        response = requests.get(url)
        if response.status_code == 200:
            with open(url.split("/")[-1], 'wb') as file:
                file.write(response.content)
        else:
            print(f"Failed to download {url}. Status code: {response.status_code}")


def construct_dataset_paths(project_name, version):
    base_path = f"{project_name}-{version}"
    train_annotation_path = f"{base_path}/train/_annotations.coco.json"
    valid_annotation_path = f"{base_path}/valid/_annotations.coco.json"
    test_annotation_path = f"{base_path}/test/_annotations.coco.json"

    train_root_dir = f"{base_path}/train"
    valid_root_dir = f"{base_path}/valid"
    test_root_dir = f"{base_path}/test"

    return train_annotation_path, valid_annotation_path, test_annotation_path, train_root_dir, valid_root_dir, test_root_dir

def create_directory(dir_path):
    """Create a directory if it does not exist."""
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)


Overwriting going_modular/utils.py


In [4]:
%%writefile going_modular/coco_dataset.py

import json
import os
import numpy as np
from PIL import Image, ImageDraw
import torch
from torchvision.transforms.v2 import functional as F
from torchvision import tv_tensors

class CustomCocoDataset(torch.utils.data.Dataset):
    def __init__(self, annotation_path, root_dir, transforms=None):
        self.root_dir = root_dir
        self.transforms = transforms

        with open(annotation_path) as f:
            self.annotations = json.load(f)

        # Filter out images without annotations
        annotated_images = []
        for img in self.annotations['images']:
            image_id = img['id']
            anns = [ann for ann in self.annotations['annotations'] if ann['image_id'] == image_id]
            if len(anns) > 0:
                annotated_images.append(img)

        self.image_ids = [img['id'] for img in annotated_images]

        # Update the self.annotations['images'] to include only annotated images
        self.annotations['images'] = annotated_images
        
        #print("Number of images:", len(self.annotations['images']))
        #print("Sample image entry:", self.annotations['images'][0])

    def __len__(self):
        return len(self.annotations['images'])

    def __getitem__(self, idx):
        img_info = self.annotations['images'][idx]
        image_id = img_info['id']
        
        img_path = os.path.join(self.root_dir, img_info['file_name'])
        img = Image.open(img_path).convert("RGB")
        img_tensor = F.to_tensor(img)
        #print("Image size (PIL):", img.size)
        #print("Image shape (tensor):", img_tensor.shape)

        anns = [ann for ann in self.annotations['annotations'] if ann['image_id'] == image_id]
        #print("Number of annotations for this image:", len(anns))

        boxes = [ann['bbox'] for ann in anns]  # bbox format: [x_min, y_min, width, height]
        # Convert from XYWH to XYXY format
        boxes = [[box[0], box[1], box[0] + box[2], box[1] + box[3]] for box in boxes]
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = [ann['category_id'] for ann in anns]
        labels = torch.as_tensor(labels, dtype=torch.int64)
        #print("Boxes shape:", boxes.shape)
        #print("Labels:", labels)
        # Debug print
        #print(f"Boxes shape for image {idx}: {boxes.shape}")

        masks = []
        for ann in anns:
            if 'segmentation' in ann and isinstance(ann['segmentation'], list):
                for seg in ann['segmentation']:
                    mask_img = Image.new('L', (img_info['width'], img_info['height']), 0)
                    ImageDraw.Draw(mask_img).polygon(seg, outline=1, fill=1)
                    mask = np.array(mask_img)
                    masks.append(mask)
        masks = torch.as_tensor(np.array(masks), dtype=torch.uint8) if masks else torch.zeros((0, img_info['height'], img_info['width']), dtype=torch.uint8)
        #print("Masks shape:", masks.shape)

        areas = [ann['area'] for ann in anns]
        areas = torch.as_tensor(areas, dtype=torch.float32)
        iscrowd = [ann['iscrowd'] for ann in anns]
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)

        # Convert masks to Mask format
        masks = tv_tensors.Mask(masks)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id  # Changed to integer
        target["area"] = areas
        target["iscrowd"] = iscrowd

        #print("Target:", target)

        if self.transforms is not None:
            img_tensor, target = self.transforms(img_tensor, target)

        return img_tensor, target





Overwriting going_modular/coco_dataset.py


In [5]:
%%writefile going_modular/visualization_utils.py
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F  # Add this import

# New function to visualize transformations
def visualize_transformation(dataset, idx):
    img, target = dataset[idx]
    transformed_img, transformed_target = dataset.transforms(img, target)
    original_img = F.to_pil_image(img)
    transformed_img = F.to_pil_image(transformed_img)

    plt.figure(figsize=(24, 6))
    # Original Image
    plt.subplot(1, 2, 1)
    plt.imshow(original_img)
    for box in target["boxes"]:
        x_min, y_min, x_max, y_max = box.tolist()
        rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='r', facecolor='none')
        plt.gca().add_patch(rect)
        #print(x_min, y_min, x_max, y_max)
    plt.title(f"Original Image - ID: {idx}")

    # Transformed Image
    plt.subplot(1, 2, 2)
    plt.imshow(transformed_img)
    for box in transformed_target["boxes"]:
        x_min, y_min, x_max, y_max = box.tolist()
        rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='b', facecolor='none')
        plt.gca().add_patch(rect)
        #print(x_min, y_min, x_max, y_max)
    plt.title(f"Transformed Image - ID: {idx}")
    plt.show()



def visualize_bbox(dataset, idx):
    img, target = dataset[idx]
    original_img = F.to_pil_image(img)

    plt.figure(figsize=(12, 6))
    plt.imshow(original_img)

    for box in target["boxes"]:  # Access the boxes directly
        x_min, y_min, x_max, y_max = box.tolist()
        # Debug print
        print(f"Visualizing BBox - xmin: {x_min}, ymin: {y_min}, xmax: {x_max}, ymax: {y_max}")
        rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='r', facecolor='none')
        plt.gca().add_patch(rect)

    plt.title(f"Image with Bounding Boxes - ID: {idx}")
    plt.show()


Overwriting going_modular/visualization_utils.py


In [6]:
%%writefile going_modular/model_utils.py
import json
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def load_classes_from_json(file_path):
    """
    Loads the class names and their corresponding IDs from a COCO format JSON file.

    Args:
    file_path (str): Path to the JSON file.

    Returns:
    dict: A dictionary where keys are class IDs and values are class names.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Extracting classes and their IDs
    classes = {category['id']: category['name'] for category in data['categories']}
    return classes

# Usage example:
#classes = load_classes_from_json('basketball_child-6/test/_annotations.coco.json')
#print(classes)

# model_utils.py
def get_model_instance_segmentation(num_classes, hidden_layer=256):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    return model



Overwriting going_modular/model_utils.py


In [7]:
%%writefile going_modular/transforms.py
import torch  # Add this import statement
from torchvision.transforms import v2 as T
from torchvision.transforms import Compose, RandomHorizontalFlip, ToTensor, ConvertImageDtype

def get_transform(train):
    transforms = []
    #if train:
    #    transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)



Overwriting going_modular/transforms.py


In [8]:
%%writefile going_modular/engine.py
# train.py
import torch
import torchvision
from engine import train_one_epoch, evaluate
from coco_utils import get_coco_api_from_dataset
from coco_eval import CocoEvaluator

def train_model(model, data_loader, data_loader_valid, device, num_epochs,
                lr=0.005, momentum=0.9, weight_decay=0.0005, step_size=3, gamma=0.1):
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=lr, momentum=momentum, weight_decay=weight_decay)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    for epoch in range(num_epochs):
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        lr_scheduler.step()
        evaluate(model, data_loader_valid, device=device)

    #torch.save(model.state_dict(), 'results/models/model_weights.pth')


Overwriting going_modular/engine.py


In [65]:
%%writefile going_modular/process_video_check.py

import cv2
import torch
import numpy as np
from torchvision.transforms import v2 as T
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks


def intersects(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1.tolist()
    x2_min, y2_min, x2_max, y2_max = box2.tolist()
    return (x1_min < x2_max and x1_max > x2_min and y1_min < y2_max and y1_max > y2_min)

def process_video_check(video_path, model, device, classes, classes_to_track, threshold=0.5):
    model.eval()
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print("Error opening video file")
        return

    score = 0  # Initialize the score counter

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_tensor = T.ToTensor()(frame).unsqueeze_(0).to(device)

        with torch.no_grad():
            prediction = model(frame_tensor)[0]

        pred_scores = prediction['scores']
        pred_boxes = prediction['boxes']
        pred_labels = prediction['labels']
        pred_masks = prediction['masks']

        keep = pred_scores > threshold
        pred_boxes = pred_boxes[keep]
        pred_labels = pred_labels[keep]
        pred_masks = pred_masks[keep]

        if not keep.any():
            continue  # Skip this frame if no detections are kept

        # Convert numeric labels to class names
        pred_class_names = [classes[label.item()] for label in pred_labels]

        # Iterate through each pair of classes to track
        for class_pair in classes_to_track:
            class1_boxes = pred_boxes[[name == class_pair[0] for name in pred_class_names]]
            class2_boxes = pred_boxes[[name == class_pair[1] for name in pred_class_names]]

            # Check for intersections and update score
            for box1 in class1_boxes:
                for box2 in class2_boxes:
                    if intersects(box1, box2):
                        score += 1
                        print(f"Intersection detected between {class_pair[0]} and {class_pair[1]}, Score:", score)

        # Frame Tensor Conversion for Drawing
        frame_tensor = (255.0 * (frame_tensor - frame_tensor.min()) / (frame_tensor.max() - frame_tensor.min())).to(torch.uint8)
        frame_tensor = frame_tensor.squeeze().to(torch.uint8)

        # Draw bounding boxes and segmentation masks
        output_image = draw_bounding_boxes(frame_tensor, pred_boxes, labels=pred_class_names, colors="red")
        output_image = draw_segmentation_masks(output_image, (pred_masks > 0.7).squeeze(1), alpha=0.5, colors="blue")

        # Convert output image for displaying
        output_image = output_image.permute(1, 2, 0).cpu().numpy().astype(np.uint8)
        output_image = np.clip(output_image, 0, 255)  # Ensure values are within 0-255

        # Draw score text
        cv2.putText(output_image, f'Score: {score}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        cv2.imshow('Frame', output_image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage
# process_video_check(video_path, model, device, classes, [('ball', 'rim')], threshold=0.5)


Overwriting going_modular/process_video_check.py


In [32]:
#testing the train.py before command line commitment

from going_modular.utils import get_device, get_project, download_files, construct_dataset_paths, download_videos_from_youtube
from going_modular.coco_dataset import CustomCocoDataset
from going_modular.visualization_utils import visualize_transformation, visualize_bbox
from going_modular.model_utils import get_model_instance_segmentation, load_classes_from_json
from going_modular.engine import train_model
from going_modular.transforms import get_transform
#from going_modular.video_processing import process_video
import torchvision.transforms as T
import utils
import torch
from torch import nn

# Note: this notebook requires torch >= 1.10.0
print(torch.__version__)

# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

data_path = 'results/data'
model_path = 'results/models'
create_directory(data_path)
create_directory(model_path)

urls = [
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py",
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py",
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py",
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py",
    "https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py"
]

download_files(urls)


# roboflow details
api_key = "htpcxp3XQh7SsgMfjJns"
workspace = "basketball-formations"#"ai-79z1a"
project_name = "basketball-and-hoop-7xk0h"#"basketball_child"
project_folder_name = "basketball-and-hoop"#"basketball_child"
version = 10#6


# Get the project dataset
dataset = get_project(api_key, workspace, project_name, version)

#use load_classes_from_json to load the classes and get length of classes
classes = load_classes_from_json(f'{project_folder_name}-{version}/test/_annotations.coco.json')
print(classes)

#automate classes volume from roboflow dataset
num_classes = len(classes)+1  # Update with the actual number of classes


# Hyperparameters and settings
lr = 0.01  # Learning rate
momentum = 0.8
weight_decay = 0.0003
step_size = 5
gamma = 0.1
hidden_layer = 512
threshold = 0.6
display_video = True  # Set to True to display the video
num_workers = 0
num_epochs = 1  # Define the number of epochs for training

# Initialize device
device = get_device()



# Construct dataset paths
train_annotation_path, valid_annotation_path, test_annotation_path, train_image_dir, valid_image_dir, test_image_dir = construct_dataset_paths(project_folder_name, version)


# Create datasets
train_dataset = CustomCocoDataset(train_annotation_path, train_image_dir, transforms=get_transform(train=True))
valid_dataset = CustomCocoDataset(valid_annotation_path, valid_image_dir, transforms=get_transform(train=False))
test_dataset = CustomCocoDataset(test_annotation_path, test_image_dir)

# Load model
model = get_model_instance_segmentation(num_classes, hidden_layer=hidden_layer)
model.to(device)

# Data Loaders
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=num_workers, collate_fn=utils.collate_fn)
valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=num_workers, collate_fn=utils.collate_fn)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=num_workers, collate_fn=utils.collate_fn)

# Optionally, visualize a sample from the dataset
sample_idx = 0  # Adjust as needed
#visualize_transformation(train_dataset, sample_idx)
 
# Train model
train_model(model, train_data_loader, valid_data_loader, 
            device, num_epochs) # Use default hyperparameters
#train_model(model, train_data_loader, valid_data_loader, device, num_epochs, lr=lr, momentum=momentum, weight_decay=weight_decay, step_size=step_size, gamma=gamma)

model_file_path = os.path.join(model_path, 'model_weights.pth')
torch.save(model.state_dict(), model_file_path)

# Process video with custom settings
video_urls = ["https://www.youtube.com/watch?v=flm-cZ7r6eE&t=8s"]
output_path = data_path
successful_downloads, failed_downloads = download_videos_from_youtube(video_urls, output_path)
#print(f"Successfully downloaded {len(successful_downloads)} videos.")
print(successful_downloads)
#print(f"Failed to download {len(failed_downloads)} videos.")
video_path = data_path + '\Stephen Curry’s Top 30 Career 3-Pointers 💦.mp4'

model_path = model_path + '\model_weights.pth'  # Replace with your model's path
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)


# Process video
#process_video(video_path, model, device, classes, threshold=threshold, ball_class_idx=1, rim_class_idx=3, display=display_video)
process_video_check(video_path, model, device, threshold=threshold)



2.1.2+cu118
loading Roboflow workspace...
loading Roboflow project...
{0: 'Basketball-and-Hoop', 1: 'Basketball', 2: 'Hoop', 3: 'Player', 4: 'backboard'}


The function `to_tensor(...)` is deprecated and will be removed in a future release. Instead, please use `to_image(...)` followed by `to_dtype(..., dtype=torch.float32, scale=True)`.


Epoch: [0]  [  0/700]  eta: 0:01:07  lr: 0.000012  loss: 7.6007 (7.6007)  loss_classifier: 2.2503 (2.2503)  loss_box_reg: 0.3824 (0.3824)  loss_mask: 2.2240 (2.2240)  loss_objectness: 2.3259 (2.3259)  loss_rpn_box_reg: 0.4180 (0.4180)  time: 0.0970  data: 0.0135  max mem: 2769
Epoch: [0]  [ 10/700]  eta: 0:01:07  lr: 0.000084  loss: 4.6103 (5.0041)  loss_classifier: 2.1039 (2.1018)  loss_box_reg: 0.3988 (0.4279)  loss_mask: 1.8406 (1.8133)  loss_objectness: 0.3307 (0.4926)  loss_rpn_box_reg: 0.1406 (0.1686)  time: 0.0972  data: 0.0123  max mem: 2769
Epoch: [0]  [ 20/700]  eta: 0:01:05  lr: 0.000155  loss: 3.4572 (4.0669)  loss_classifier: 1.7450 (1.7027)  loss_box_reg: 0.3989 (0.4279)  loss_mask: 1.4272 (1.4522)  loss_objectness: 0.1818 (0.3370)  loss_rpn_box_reg: 0.1193 (0.1471)  time: 0.0962  data: 0.0119  max mem: 2769
Epoch: [0]  [ 30/700]  eta: 0:01:04  lr: 0.000227  loss: 2.3992 (3.3397)  loss_classifier: 0.7568 (1.3218)  loss_box_reg: 0.3842 (0.4188)  loss_mask: 0.7351 (1.2069) 

In [66]:
%%writefile train.py
import argparse
import os
from going_modular.utils import (get_device, create_directory, get_project,
                                 download_files, construct_dataset_paths,
                                 download_videos_from_youtube)
from going_modular.coco_dataset import CustomCocoDataset
from going_modular.model_utils import (get_model_instance_segmentation,
                                       load_classes_from_json)
from going_modular.engine import train_model
from going_modular.transforms import get_transform
from going_modular.process_video_check import process_video_check
import utils
import torch
from torch import nn
from pathlib import Path

# Note: this notebook requires torch >= 1.10.0
print(torch.__version__)

# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

def main(args):
    # Create directories for data and models
    data_path = Path(args.data_path)
    model_path = Path(args.model_path)
    data_path.mkdir(parents=True, exist_ok=True)
    model_path.mkdir(parents=True, exist_ok=True)

    # Get the project dataset
    dataset = get_project(args.api_key, args.workspace, args.project_name, args.version)

    # Load classes from json
    classes = load_classes_from_json(f'{args.project_folder_name}-{args.version}/test/_annotations.coco.json')
    print(classes)

    # Set the number of classes
    num_classes = len(classes) + 1

    # Initialize device
    device = get_device()

    # Construct dataset paths
    train_annotation_path, valid_annotation_path, test_annotation_path, train_image_dir, valid_image_dir, test_image_dir = construct_dataset_paths(args.project_folder_name, args.version)

    # Create datasets with transforms
    train_dataset = CustomCocoDataset(train_annotation_path, train_image_dir, transforms=get_transform(train=True))
    valid_dataset = CustomCocoDataset(valid_annotation_path, valid_image_dir, transforms=get_transform(train=False))
    test_dataset = CustomCocoDataset(test_annotation_path, test_image_dir)

    # Load model
    model = get_model_instance_segmentation(num_classes, 
                                            hidden_layer=args.hidden_layer)
    model.to(device)

    # Data Loaders
    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0, collate_fn=utils.collate_fn)
    valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=0, collate_fn=utils.collate_fn)
    test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0, collate_fn=utils.collate_fn)

    # Train the model
    train_model(model, train_data_loader, valid_data_loader, device, args.num_epochs, lr=args.lr)

    # Save model after training
    model_file_path = model_path / 'model_weights.pth'
    try:
        torch.save(model.state_dict(), str(model_file_path))
        print("Model saved at:", model_file_path)
    except Exception as e:
        print(f"Error saving model: {e}")

    # Download and process video
    try:
        successful_downloads, failed_downloads = download_videos_from_youtube([args.video_url], args.data_path)
        print(f"Successfully downloaded {len(successful_downloads)} videos.")
        print(f"Failed to download {len(failed_downloads)} videos.")

        if successful_downloads:
            # Construct video path
            video_filename = args.video_name
            if not video_filename.endswith('.mp4'):
                video_filename += '.mp4'
            video_path = data_path / video_filename
            print(f"Video path: {video_path}")

            # Load model state for video processing
            model.load_state_dict(torch.load(str(model_file_path), map_location=device))
            model.to(device)

            # Process video
            process_video_check(video_path, model, device, classes, [('Basketball', 'Hoop')], threshold=args.threshold)
        else:
            print("No videos were downloaded.")

    except Exception as e:
        print(f"Error in video downloading or processing: {e}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Train a model for object detection")
    # Define default values
    default_api_key = "your_roboflow_api_key"
    default_workspace = "your_roboflow_workspace"
    default_project_name = "your_roboflow_project"
    default_project_folder_name = "your_roboflow_project_folder_name"
    default_version = 1
    default_hidden_layer = 256
    default_lr = 0.005
    default_num_epochs = 10
    default_video_url = "https://www.youtube.com/watch?v=example_video_id"
    default_video_name = "your_youtube_video_name"
    default_confidence_threshold = 0.6
    default_display_video = True
    default_data_path = 'results/data'
    default_model_path = 'results/models'

    # Setup argparse with defaults
    parser.add_argument('--api_key', type=str, default=default_api_key, help='API key for Roboflow')
    parser.add_argument('--workspace', type=str, default=default_workspace, help='Workspace name in Roboflow')
    parser.add_argument('--project_name', type=str, default=default_project_name, help='Project name in Roboflow')
    parser.add_argument('--project_folder_name', type=str, default=default_project_folder_name, help='Project folder name in Roboflow')
    parser.add_argument('--version', type=int, default=default_version, help='Version of the dataset in Roboflow')
    parser.add_argument('--hidden_layer', type=int, default=default_hidden_layer, help='Hidden layer size for the MaskRCNN predictor')
    parser.add_argument('--lr', type=float, default=default_lr, help='Learning rate')
    parser.add_argument('--num_epochs', type=int, default=default_num_epochs, help='Number of epochs to train the model')
    parser.add_argument('--video_url', type=str, default=default_video_url, help='URL of the video to process')
    parser.add_argument('--video_name', type=str, default=default_video_name, help='Name of the video file (with extension) to process')
    parser.add_argument('--threshold', type=float, default=default_confidence_threshold, help='Detection threshold for process_video')
    parser.add_argument('--display_video', type=bool, default=default_display_video, help='Whether to display the video during processing')
    parser.add_argument('--data_path', type=str, default=default_data_path, help='Path to save downloaded data')
    parser.add_argument('--model_path', type=str, default=default_model_path, help='Path to save model weights')

    args = parser.parse_args()
    main(args)


Overwriting train.py


In [68]:
#example usage
!python train.py --api_key htpcxp3XQh7SsgMfjJns \
                --workspace basketball-formations \
                --project_name basketball-and-hoop-7xk0h \
                --project_folder_name basketball-and-hoop \
                --version 10 \
                --hidden_layer 256 \
                --lr 0.005 \
                --num_epochs 10 \
                --threshold 0.6 \
                --video_url "https://www.youtube.com/watch?v=y8i6fsAXDZE" \
                --video_name "The Best NBA 3 Point Contest Performances"



2.1.2+cu118

loading Roboflow workspace...

loading Roboflow project...
{0: 'Basketball-and-Hoop', 1: 'Basketball', 2: 'Hoop', 3: 'Player', 4: 'backboard'}
Epoch: [0]  [  0/700]  eta: 0:08:25  lr: 0.000012  loss: 3.8863 (3.8863)  loss_classifier: 1.7396 (1.7396)  loss_box_reg: 0.4441 (0.4441)  loss_mask: 1.0195 (1.0195)  loss_objectness: 0.3417 (0.3417)  loss_rpn_box_reg: 0.3413 (0.3413)  time: 0.7215  data: 0.0165  max mem: 1670
Epoch: [0]  [ 10/700]  eta: 0:01:50  lr: 0.000084  loss: 3.3805 (3.4668)  loss_classifier: 1.7071 (1.6481)  loss_box_reg: 0.3912 (0.4308)  loss_mask: 1.0936 (1.1141)  loss_objectness: 0.1390 (0.1668)  loss_rpn_box_reg: 0.0486 (0.1071)  time: 0.1604  data: 0.0119  max mem: 2154
Epoch: [0]  [ 20/700]  eta: 0:01:30  lr: 0.000155  loss: 2.8309 (2.9953)  loss_classifier: 1.3272 (1.3184)  loss_box_reg: 0.4027 (0.4504)  loss_mask: 0.8310 (0.9296)  loss_objectness: 0.1045 (0.1814)  loss_rpn_box_reg: 0.0486 (0.1156)  time: 0.1039  data: 0.0111  max mem: 2154
Epoch: [0]

The function `to_tensor(...)` is deprecated and will be removed in a future release. Instead, please use `to_image(...)` followed by `to_dtype(..., dtype=torch.float32, scale=True)`.
The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
